In [47]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Tải dữ liệu giá và SAR
gold = yf.download('GC=F', '2022-03-01', '2024-03-01', auto_adjust=True)

def calculate_sar(df, acceleration=0.02, maximum=0.2):
    high = df['High']
    low = df['Low']
    close = df['Close']
    sar = pd.Series(index=df.index)
    sar[0] = low[0]
    trend = 0
    ep = 0
    af = acceleration
    for i in range(1, len(df)):
        if trend == 0:
            if high[i - 1] > high[i] and low[i - 1] > low[i]:
                trend = 1
                sar[i] = low[i]
                ep = high[i]
                af = acceleration
            elif high[i - 1] < high[i] and low[i - 1] < low[i]:
                trend = -1
                sar[i] = high[i]
                ep = low[i]
                af = acceleration
        else:
            if trend == 1:
                if high[i] > ep:
                    sar[i] = sar[i - 1] + af * (ep - sar[i - 1])
                    sar[i] = min(sar[i], low[i - 1])
                    ep = high[i]
                    af = min(af + acceleration, maximum)
                else:
                    trend = -1
                    sar[i] = ep
                    sar[i] = min(sar[i], low[i - 1])
                    sar[i] = min(sar[i], low[i])
                    ep = low[i]
                    af = acceleration
            elif trend == -1:
                if low[i] < ep:
                    sar[i] = sar[i - 1] + af * (ep - sar[i - 1])
                    sar[i] = max(sar[i], high[i - 1])
                    ep = low[i]
                    af = min(af + acceleration, maximum)
                else:
                    trend = 1
                    sar[i] = ep
                    sar[i] = max(sar[i], high[i - 1])
                    sar[i] = max(sar[i], high[i])
                    ep = high[i]
                    af = acceleration
    return sar

# Tính toán SAR cho dữ liệu và thêm vào DataFrame
gold['SAR'] = calculate_sar(gold)

# Chuẩn bị dữ liệu cho mô hình LSTM
data = gold[['Close', 'SAR']].copy()
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Hàm tạo dữ liệu cho mô hình LSTM
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:(i + look_back), :])
        Y.append(dataset[i + look_back, 1])
    return np.array(X), np.array(Y)

# Thiết lập siêu tham số
look_back = 1
batch_size = 32
epochs = 100

# Tạo dữ liệu train và test
train_size = int(len(scaled_data) * 0.8)
test_size = len(scaled_data) - train_size
train, test = scaled_data[0:train_size, :], scaled_data[train_size:len(scaled_data), :]
X_train, Y_train = create_dataset(train, look_back)
X_test, Y_test = create_dataset(test, look_back)

# Reshape dữ liệu
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 2))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 2))

# Xây dựng mô hình LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(look_back, 2)))
model.add(LSTM(units=50))
model.add(Dense(units=1))

# Compile mô hình
model.compile(optimizer='adam', loss='mean_squared_error')

# Fit mô hình
model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, verbose=2)

# Dự đoán SAR cho 60 ngày tiếp theo
future_data = scaled_data[-look_back:, :]
X_future = np.reshape(future_data, (1, look_back, 2))
predicted_sar = model.predict(X_future)




[*********************100%***********************]  1 of 1 completed


C:\Users\My My\AppData\Local\Temp\ipykernel_22688\2768380803.py:15: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  sar = pd.Series(index=df.index)


Epoch 1/100
13/13 - 7s - loss: 0.2858 - 7s/epoch - 525ms/step
Epoch 2/100
13/13 - 0s - loss: 0.2077 - 54ms/epoch - 4ms/step
Epoch 3/100
13/13 - 0s - loss: 0.1140 - 50ms/epoch - 4ms/step
Epoch 4/100
13/13 - 0s - loss: 0.0332 - 51ms/epoch - 4ms/step
Epoch 5/100
13/13 - 0s - loss: 0.0149 - 47ms/epoch - 4ms/step
Epoch 6/100
13/13 - 0s - loss: 0.0130 - 47ms/epoch - 4ms/step
Epoch 7/100
13/13 - 0s - loss: 0.0111 - 47ms/epoch - 4ms/step
Epoch 8/100
13/13 - 0s - loss: 0.0096 - 47ms/epoch - 4ms/step
Epoch 9/100
13/13 - 0s - loss: 0.0084 - 47ms/epoch - 4ms/step
Epoch 10/100
13/13 - 0s - loss: 0.0073 - 92ms/epoch - 7ms/step
Epoch 11/100
13/13 - 0s - loss: 0.0063 - 51ms/epoch - 4ms/step
Epoch 12/100
13/13 - 0s - loss: 0.0053 - 47ms/epoch - 4ms/step
Epoch 13/100
13/13 - 0s - loss: 0.0046 - 52ms/epoch - 4ms/step
Epoch 14/100
13/13 - 0s - loss: 0.0039 - 48ms/epoch - 4ms/step
Epoch 15/100
13/13 - 0s - loss: 0.0034 - 44ms/epoch - 3ms/step
Epoch 16/100
13/13 - 0s - loss: 0.0030 - 46ms/epoch - 4ms/step
E

In [48]:
# # Chuyển ngược quá trình scaling để lấy giá trị SAR thực tế
# predicted_sar_scaled = np.concatenate((future_data[:, -1].reshape(-1, 1), predicted_sar.reshape(-1, 1)), axis=1)
# predicted_sar = scaler.inverse_transform(predicted_sar_scaled)[:, -1]

# In ra dự đoán SAR cho 60 ngày tiếp theo
print(predicted_sar)


[[0.90139824]]
